# Pricing of options with CRR tree and constant implied volatility

### *Objectives*: Using the Cox-Ross-Rubinstein tree, calculation of european, american and other exotic (non complex) options.  

We calculate the values of the options using the CRR tree, and compare them (when possible) with values calculated using the Black and Scholes formula.   

*As a reminder* : using implied volatility does not permit the correct pricing of options observed in the market; local volatility will be used.

In [129]:
"""
pricer_CRR_tree_constant_implied_vol_for_options.ipynb
======================================================
This script calculates pricing call and put option for a given set of parameters.

Dependencies:
- `numpy`: For numerical operations.
- `math`: For mathematical operations.
- `scipy.stats`: For statistical functions, specifically the normal distribution.
"""

import numpy as np  

import math

from scipy.stats import norm

## Calculate the values in a binomial tree.  

### Calculate the values of a stock. 

At each node we calculate the value of a stock using the Cox-Ross-Rubinstein model :

$ S(t_n) = S(t_{n-1}) * e^{\sigma \sqrt{\Delta t}}$ $ \Rightarrow $ when the value goes up.    
$ S(t_n) = S(t_{n-1}) * e^{-\sigma \sqrt{\Delta t}}$ $ \Rightarrow $ when the value goes down.  

*where*:    
$e^{\sigma \sqrt{ \Delta t}}$ = u (*in script below*).   
$e^{-\sigma \sqrt{ \Delta t}}$ = d (*in script below*).       

Therefore the amplitude of the movement of the value going up or down, depends on the implied volatility of the stock.   
We consider the Black and Scholes implied volatility, so that we can compare the CRR tree method and Black and Scholes model.  


In [130]:
def crr_tree(S0, T, r, q, sigma, n_steps):
    """
    Calculate the prices of a stock at each node of a binomial tree using the Cox-Ross-Rubinstein (CRR) model.
        
    Args:
        S0 (float): Current stock price.
        T (float): Time to maturity in years.
        r (float): Risk-free interest rate.
        q (float): Dividend yield.
        sigma (float): Volatility of the underlying asset.
        n_steps (int): Number of time steps in the binomial tree.
        
     Returns:
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        The first dimension represents the time steps, and the second dimension represents the possible stock prices at each step.  
        p (float): Risk-neutral probability of an upward movement.
        discount_factor (float): Discount factor per time step.
    """

    dt = T / n_steps # Time step size
    u = math.exp(sigma * math.sqrt(dt)) # Up factor
    d = 1 / u   # Down factor

    p = (math.exp((r-q)*dt) - d) / (u - d) # Risk-neutral probability
    discount_factor = math.exp(-r * dt)  # Discount factor per time step

    # Initialize asset prices at maturity
    S = np.zeros((n_steps + 1, n_steps + 1))    
    S[0,0] = S0 # Initial stock price

    # At each step, calculate the stock prices
    for i in range(1, n_steps + 1) :
        S[:i,i] = S[:i,i-1] * u
        S[i,i] = S[i-1,i-1] * d

    return S, p, discount_factor

### Calculate the values of an option. 

We calculate the value of the option at each node based on the previously calculated price of a stock.   
The formula of the value of the option depends on the type of option *(see below)*.  


In [131]:
def tree_option_pricing(payoff, S, K, p, discount_factor, n_steps, B=None, option_type=None):
    """
    Calculate the option prices at each node of a binomial tree using the Cox-Ross-Rubinstein (CRR) model.
    
    Args:
        payoff (function): Function to calculate the option payoff at maturity.
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        p (float): Risk-neutral probability of an upward movement.
        discount_factor (float): Discount factor per time step.
        n_steps (int): Number of time steps in the binomial tree.
        H (float, optional): Barrier level for barrier options. Default is None.
        option_type (str, optional): Type of option ('call' or 'put'). Default is None.
    
    Returns:
        tuple: Call and put option prices.
    """
    
    v = np.zeros(n_steps + 1)  # Initialize option prices at maturity
    if (option_type == 'up_and_out_call' or option_type == 'up_and_out_put' 
        or option_type == 'down_and_out_call' or option_type == 'down_and_out_put'):
        v = payoff(n_steps, v, S, K, B, n_steps)  # Payoff at maturity
    else:
        v = payoff(n_steps, v, S, K, n_steps)    

    for i in range(n_steps - 1, -1, -1):  # Iterate backwards through the tree
        n_nodes = i + 1  # Number of nodes at this step 
        v[:n_nodes] = discount_factor * (p * v[:n_nodes] + (1-p) * v[1:n_nodes+1])  # Discount the option prices

        if (option_type == 'up_and_out_call' or option_type == 'up_and_out_put' 
        or option_type == 'down_and_out_call' or option_type == 'down_and_out_put'):
            v[:n_nodes] = payoff(i, v[:n_nodes], S, K, B, n_steps)  # Calculate the option prices at this step   
        else:
            v[:n_nodes] = payoff(i, v[:n_nodes], S, K, n_steps)

    return v[0]  # Return the option price at the root node, i.e. at the current time

## Calculate the value of an option.

### Calculate the value of an european call option.  

From the final prices (at maturity T) of a stock i.e. at the last nodes, we calculate the possible payoffs of the option.  

$C_T = max(S_T - K; 0)$ 

In [132]:
def european_call_payoff(i, v_i, S, K, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for a European call option.
    
    Args:
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        n_steps (int): Total number of time steps in the binomial tree.  
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    if i == n_steps:
        v_i = np.maximum(S[:, n_steps] - K, 0)

    return v_i    

### Calculate the value of an european put option.  

From the final prices (at maturity T) of a stock i.e. at the last nodes, we calculate the possible payoffs of the option.  

$P_T = max(K - S_T; 0)$


In [133]:
def european_put_payoff(i, v_i, S, K, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for a European put option.
    
    Args:
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        n_steps (int): Total number of time steps in the binomial tree.  
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    if i == n_steps:
        v_i = np.maximum(K - S[:, n_steps], 0)

    return v_i

### Calculate the value of an american call option.  

An american option can be triggered at each node and therefore the calculated value is compared to the value of the call option as if the option terminates now. 
 
Using the probabilities to go up and down at each node $t+\Delta t$, we calculate the value of the previous node $t$, until the 1st node (i.e. we calculate the nodes, backwards until the current time).   

$V_{t,k} = max (S_{t,k} - K, e^{-r\Delta t} (p V_{t+\Delta t,k} + (1-p)V_{t+\Delta t,k+1}))$.  

*with*:   
$V_{t,k} = C_t = max(S_t - K; 0)$
 
$p = \frac{e^{(r-q)\Delta t} - d}{u - d}$ $ \Rightarrow $ the risk-neutral probability of a stock to go up or down (*in script above*). 


In [134]:
def american_call_payoff(i, v_i, S, K, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for an American call option.
    
    Args:
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option. 
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    n_nodes = i + 1  # Number of nodes at this step
    return np.maximum(S[:n_nodes, i] - K, v_i)  # Calculate the option prices at this step

### Calculate the value of an american put option.  

As the american call option, the calculated value is compared to the value of the put option as if the option terminates now.

$V_{t,k} = max (K - S_{t,k}, e^{-r\Delta t} (p V_{t+\Delta t,k} + (1-p)V_{t+\Delta t,k+1}))$.  

*with*:   
$V_{t,k} = P_t = max(K - S_t; 0)$

In [135]:
def american_put_payoff(i, v_i, S, K, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for an American put option.
    
    Args:
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    n_nodes = i + 1  # Number of nodes at this step
    return np.maximum(K - S[:n_nodes, i], v_i)  # Calculate the option prices at this step

### Calculate the value of an up and out call option.  

For option with barrier, options can be activated or deactivated (independantly from the owner of the option) if the underlying satisfies a condition.

Here the call option is deactivated if the underlying crosses a barrier higher than the strike.

$V_{t,k} = e^{-r\Delta t} (p V_{t+\Delta t,k} + (1-p)V_{t+\Delta t,k+1}) \mathbf{1}_{S_{t,k} < B}$.  

*with*:   
$V_{T,k} = C_T = max(S_T - K; 0)$


In [136]:
def up_and_out_call_payoff(i, v_i, S, K, H, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for an up-and-out call option.
    
    Args:
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        H (float): Barrier level for the up-and-out option.
        n_steps (int): Total number of time steps in the binomial tree.  

    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    if i == n_steps:
        v_i = np.maximum(S[:, n_steps] - K, 0)
        
    n_nodes = i + 1  # Number of nodes at this step
    v_i = np.where(S[:n_nodes, i] < H, v_i, 0) # Set option price to 0 if the stock price is above the barrier level

    return v_i

### Calculate the value of an up and out put option.  

Here the put option is deactivated if the underlying crosses a barrier higher than the strike.

$V_{t,k} = e^{-r\Delta t} (p V_{t+\Delta t,k} + (1-p)V_{t+\Delta t,k+1}) \mathbf{1}_{S_{t,k} < B}$.  

*with*:   
$V_{T,k} = P_T = max(K - S_T; 0)$

In [137]:
def up_and_out_put_payoff(i, v_i, S, K, H, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for an up-and-out put option.
    
    Args:
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        H (float): Barrier level for the up-and-out option.
        n_steps (int): Total number of time steps in the binomial tree.  
 
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    if i == n_steps:
        v_i = np.maximum(K - S[:, n_steps], 0)
        
    n_nodes = i + 1  # Number of nodes at this step
    v_i = np.where(S[:n_nodes, i] < H, v_i, 0) # Set option price to 0 if the stock price is above the barrier level

    return v_i

### Calculate the value of a down and out call option.  

Here the call option is deactivated if the underlying crosses a barrier lower than the strike.

$V_{t,k} = e^{-r\Delta t} (p V_{t+\Delta t,k} + (1-p)V_{t+\Delta t,k+1}) \mathbf{1}_{S_{t,k} > B}$.  

*with*:   
$V_{T,k} = C_T = max(S_T - K; 0)$

In [138]:
def down_and_out_call_payoff(i, v_i, S, K, L, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for a down-and-out option.
    
    Args:
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        H (float): Barrier level for the down-and-out option.
        i (int): Current time step in the binomial tree.
        
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    if i == n_steps:
        v_i = np.maximum(S[:, n_steps] - K, 0)
        
    n_nodes = i + 1  # Number of nodes at this step
    v_i = np.where(S[:n_nodes, i] > L, v_i, 0) # Set option price to 0 if the stock price is below the barrier level

    return v_i

### Calculate the value of a down and out put option.  

Here the put option is deactivated if the underlying crosses a barrier lower than the strike.

$V_{t,k} = e^{-r\Delta t} (p V_{t+\Delta t,k} + (1-p)V_{t+\Delta t,k+1}) \mathbf{1}_{S_{t,k} > B}$.  

*with*:   
$V_{T,k} = P_T = max(K - S_T; 0)$

In [139]:
def down_and_out_put_payoff(i, v_i, S, K, L, n_steps):
    """
    Calculate the option prices at each node of the binomial tree for a down-and-out put option.
    
    Args:
        i (int): Current time step in the binomial tree.
        v_i (numpy.ndarray): Array of stock prices at the current time step.
        S (numpy.ndarray): A 2D array representing the stock prices at each node of the binomial tree.
        K (float): Strike price of the option.
        H (float): Barrier level for the down-and-out option.
        i (int): Current time step in the binomial tree.
        
    Returns:
        v_i (numpy.ndarray): Updated array of option prices at the current time step.
    """
    if i == n_steps:
        v_i = np.maximum(K - S[:, n_steps], 0)
        
    n_nodes = i + 1  # Number of nodes at this step
    v_i = np.where(S[:n_nodes, i] > L, v_i, 0) # Set option price to 0 if the stock price is below the barrier level

    return v_i

## Calculate the value of an european option with the Black-Scholes formula

### Calculate the value of a call option with the Black-Scholes formula

$C_0 = S_0\mathcal{N}(d1) - Ke^{-rT}\mathcal{N}(d2)$

*with*:   
$d1 = \frac{log\frac{S_0}{K}+(r+\frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$    

$d2 = d1 - \sigma \sqrt{T}$

In [ ]:
def bs_call(S, K, T, sigma, r):
    """
    Calculate the Black-Scholes price of a European call option.
    
    Args:
        S (float): Current stock price.
        K (float): Strike price of the option.
        T (float): Time to maturity in years.
        sigma (float): Volatility of the underlying asset.
        
    Returns:
        float: Black-Scholes price of the European call option.
    """
    if T == 0 or sigma == 0:
        return max(S - K, 0)
    else:
        d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)  

### Calculate the value of a put option with the Black-Scholes formula

$P_0 = Ke^{-rT}\mathcal{N}(-d2) - S_0\mathcal{N}(-d1) $

In [141]:
def bs_put(S, K, T, sigma, r):
    """
    Calculate the Black-Scholes price of a European put option.
    
    Args:
        S (float): Current stock price.
        K (float): Strike price of the option.
        T (float): Time to maturity in years.
        sigma (float): Volatility of the underlying asset.
        
    Returns:
        float: Black-Scholes price of the European put option.
    """
    if T == 0 or sigma == 0:
        return max(K - S, 0)
    else:
        d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        
        return -S * norm.cdf(-d1) + K * math.exp(-r * T) * norm.cdf(-d2)   

### Calculate the value of an european option with the Black-Scholes formula

Using the Black Scholes formula, the value of an european option should converge with the value of an european option calculated with the CRR tree.   

In [142]:
def calculate_bs_option(bs_payoff, S, K, T, sigma, r, q):
    """
    Calculate the Black-Scholes price of an option.
    
    Args:
        payoff (function): Function to calculate the option payoff.
        S (float): Current stock price.
        K (float): Strike price of the option.
        T (float): Time to maturity in years.
        sigma (float): Volatility of the underlying asset.
        
    Returns:
        float: Black-Scholes price of the option.
    """
    F = S * math.exp((r - q) * T)  # Adjusted stock price for risk-neutral valuation
    return bs_payoff(F, K, T, sigma, 0) * math.exp(-r * T)  # Discount the option price to present value

## Execution of the project

In [143]:
def main():
    """
    Main function to execute the CRR tree option pricing.
    This function sets up the parameters for the option pricing and calls the crr_tree function.
    It then prints the calculated call and put option prices.
    """

    # Input parameters for underlying asset and option
    S0 = 100  # Current stock price
    K = 100  # Strike price
    r = 0.015 # Risk-free interest rate
    q = 0. # Dividend yield
    sigma = 0.2 # Volatility of the underlying asset

    H = 110  # High Barrier level for the up-and-out option
    L = 90  # Low Barrier level for the down-and-out option

    # Input parameters for option pricing / binomial tree
    T = 1    # Time to maturity in years
    n_steps = 750  # Number of time steps


    # Calculate the stock prices using the CRR tree
    S, p, discount_factor = crr_tree(S0, T, r, q, sigma, n_steps)   

    # Calculate the option prices using the CRR tree
    european_call = tree_option_pricing(european_call_payoff, S, K, p, discount_factor, n_steps)
    american_call = tree_option_pricing(american_call_payoff, S, K, p, discount_factor, n_steps)
    european_put = tree_option_pricing(european_put_payoff, S, K, p, discount_factor, n_steps)
    american_put = tree_option_pricing(american_put_payoff, S, K, p, discount_factor, n_steps)
    up_and_out_call = tree_option_pricing(up_and_out_call_payoff, S, K, p, discount_factor, n_steps, H, option_type='up_and_out_call')
    up_and_out_put = tree_option_pricing(up_and_out_put_payoff, S, K, p, discount_factor, n_steps, H, option_type='up_and_out_put')
    down_and_out_call = tree_option_pricing(down_and_out_call_payoff, S, K, p, discount_factor, n_steps, L, option_type= 'down_and_out_call')   
    down_and_out_put = tree_option_pricing(down_and_out_put_payoff, S, K, p, discount_factor, n_steps, L, option_type='down_and_out_put')

    # Calculate the Black-Scholes prices for comparison
    european_bs_call = calculate_bs_option(bs_call, S0, K, T, sigma, r, q)
    european_bs_put = calculate_bs_option(bs_put, S0, K, T, sigma, r, q)
    
    # Print the calculated option prices
    print("European call = {:.4f} (BS = {:.4f})".format(european_call, european_bs_call))
    print("American call = {:.4f}".format(american_call))
    print("European put = {:.4f}(BS = {:.4f})".format(european_put, european_bs_put))
    print("American put = {:.4f}".format(american_put))
    print("Up-and-out call = {:.4f}".format(up_and_out_call))
    print("Up-and-out put = {:.4f}".format(up_and_out_put))
    print("Down-and-out call = {:.4f}".format(down_and_out_call))
    print("Down-and-out put = {:.4f}".format(down_and_out_put))


In [144]:
if __name__ == "__main__":
    main()

S= 101.51130646157189
K= 100
r= 0
sigma= 0.2
T= 1
d1= 0.17499999999999977
d2= -0.025000000000000244
European call = 8.6702 (BS = 8.6728)
American call = 8.6702
European put = 7.1814(BS = 7.1840)
American put = 7.3056
Up-and-out call = 0.1512
Up-and-out put = 5.7100
Down-and-out call = 7.2119
Down-and-out put = 0.1900
